In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_tensor, to_pil_image
import torch.optim as optim
from PIL import Image, ImageDraw, ImageFont
import json
import random
import gradio as gr

import clip
import os
from tqdm import tqdm
from utils.transform_image import draw_text_with_new_lines, generate_all_fonts_embedded_images
from utils.init_model import load_model, preprocess
from utils.initialize_font_data import retrieve_font_path, fox_text, fox_text_four_lines
from utils.cross_lingual_font_retrieval import inclusive_cjk_font_paths

from cj_fonts import inclusive_fonts, fifty_fonts
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager
from matplotlib.font_manager import FontProperties

# If using GPU then use mixed precision training.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Must set jit=False for training
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

exclusive_attributes:  ['capitals', 'cursive', 'display', 'italic', 'monospace', 'serif']


In [2]:
font_dir = '../gwfonts'
cj_font_dir = '../all-fonts'
char_size = 150
fifty_font_paths = [os.path.join(cj_font_dir, f) for f in fifty_fonts.split('\n') if f != '']

# add font
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

for font in font_manager.findSystemFonts(cj_font_dir):
    font_manager.fontManager.addfont(font)

ttf_list = font_manager.fontManager.ttflist

all_json_path = '../attributeData/all_font_to_attribute_values.json'
train_json_path = '../attributeData/train_font_to_attribute_values.json'
test_json_path = '../attributeData/test_font_to_attribute_values.json'
validation_json_path = '../attributeData/validation_font_to_attribute_values.json'
all_json = json.load(open(all_json_path, 'r'))
train_json = json.load(open(train_json_path, 'r'))
test_json = json.load(open(test_json_path, 'r'))
validation_json = json.load(open(validation_json_path, 'r'))
train_font_names = list(train_json.keys())
test_font_names = list(test_json.keys())
validation_font_names = list(validation_json.keys())

font_names = list(all_json.keys())
font_paths = [retrieve_font_path(font_name, font_dir=font_dir) for font_name in font_names]
cj_font_paths = sorted([os.path.join(cj_font_dir, tmp_font_path) for tmp_font_path in os.listdir(cj_font_dir)])
all_gwfont_paths = sorted([os.path.join(font_dir, tmp_font_path) for tmp_font_path in os.listdir(font_dir)])
#font_names = [os.path.splitext(os.path.basename(f))[0] for f in os.listdir(font_dir)]

In [6]:
checkpoint_path = 'model_checkpoints/91011_1011_multiple_3_1000_aug_ex.pt'
checkpoint_path = 'model_checkpoints/91011_1011_multiple_3_1000_image_file_dir_aug.pt'
checkpoint_path = 'model_checkpoints/best_ViT-B_32_cnn_based_vae_loss_weight_3.0_vae_loss_kl_weight0.001_res_64_9101191011_batch64_aug200_lower_bound_of_scale0.35_use_negative_lr1e-05-0.1.pt'
# checkpoint_path = 'model_checkpoints/leave_one_out_angular_ViT-B_32_9101191011_batch64_aug250_lower_bound_of_scale0.35_use_negative_lr2e-05-0.1_image_file_dir.pt'
# checkpoint_path = 'model_checkpoints/2_ViT-B_32_9101191011_batch64_aug250_use_negative_use_negative_loss0.0_lr2e-05-0.1_image_file_dir.pt'
#checkpoint_path = 'model_checkpoints/new_best_fox_negative_91011_1011_use_weight_image_file_dir_ex.pt'
# checkpoint_path = 'model_checkpoints/best_ViT-B_32_cnn_based_vae_loss_weight_3.0_vae_loss_kl_weight0.001_res_64_9101191011_batch64_aug200_lower_bound_of_scale0.35_use_negative_lr1e-05-0.1.pt'
# checkpoint_path = None
model = load_model(model, checkpoint_path)
image_file_dir = None
# image_file_dir = '../attributeData/grayscale_images'
#text = 'SIGGRAPH\nASIA'
#text = 'シーグラフ\nアジア'
# text = '春夏\n秋冬'
#text = '木材'
# text = '夏林\n火山'
text = fox_text_four_lines
text = '夏林\n火山'
# text = '夏林火\n山夏林\n火山夏'
# text = 'ユーログラ\nフィックス'
# target_font_paths = fifty_font_paths
target_font_paths = all_gwfont_paths
# target_font_paths = font_paths
# target_font_paths = cj_font_paths
target_font_paths = inclusive_cjk_font_paths
embedded_images = generate_all_fonts_embedded_images(target_font_paths, text, image_file_dir=image_file_dir, model=model, preprocess=preprocess)
embedded_images_numpy = torch.cat(list(embedded_images.values())).cpu().numpy()
font_db = embedded_images_numpy
print(len(font_db))

294


In [7]:
def create_image(text, font, char_size=char_size):
    line_num = text.count('\n') + 1
    width = int(char_size * len(text) * 1.8 / line_num)
    height = int(char_size * 1.5) * line_num
    image = draw_text_with_new_lines(text, font, width, height)
    return image


def calc_cos_sim(a, b):
    dot_product = np.dot(b, a.T)
    a_norm = np.linalg.norm(a)
    b_norm = np.linalg.norm(b, axis=1)
    cos_sim = dot_product / np.outer(b_norm, a_norm)
    #sim = dot_product / (b_norm[:, np.newaxis] * a_norm)
    return cos_sim[:, 0]

def query_text(text, font_db, model):
    input_texts = text
    if isinstance(text, str):
        input_texts = [text]
    tokenized_text = clip.tokenize(input_texts).to(device)
    embedded_text = model.encode_text(tokenized_text).cpu().numpy()
    # retrieve most similar font
    cos_sim = calc_cos_sim(embedded_text, font_db)
    sorted_index = np.argsort(-cos_sim)
    return sorted_index, cos_sim

def query_image(image, font_db, model, preprocess=preprocess):
    if isinstance(image, str):
        image = Image.open(image)
    
    preprocessed_image = preprocess(image).unsqueeze(0).to(device)
    embedded_image = model.encode_image(preprocessed_image).cpu().numpy()
    # retrieve most similar font
    cos_sim = calc_cos_sim(embedded_image, font_db)
    sorted_index = np.argsort(-cos_sim)
    return sorted_index, cos_sim

def query_image_and_text(image, text, alpha=0.5, font_db=font_db, model=model, preprocess=preprocess):
    input_texts = text
    if isinstance(text, str):
        input_texts = [text]
    tokenized_text = clip.tokenize(input_texts).to(device)
    embedded_text = model.encode_text(tokenized_text).cpu().numpy()
    preprocessed_image = preprocess(image).unsqueeze(0).to(device)
    embedded_image = model.encode_image(preprocessed_image).cpu().numpy()
    sum_embedded = alpha * embedded_image + (1 - alpha) * embedded_text
    cos_sim = calc_cos_sim(sum_embedded, font_db)
    sorted_index = np.argsort(-cos_sim)
    return sorted_index, cos_sim


In [8]:
font_images_num = 2
def save_output_buidler(*images):
    for i, image in enumerate(images):
        if image is None:
            continue
        image.save(f'output_images/{i}.png')


default_text_value = 'Eurographics'
# default_text_value = '安全出行'
#default_text_value = 'Sushi.\nStill your best bet\nfor intestinal worms.'
sorted_index = None
current_index = 0
column_num = 2
row_num = 5
def builder_query(prompt, image, alpha=1.0, sample_text='hand write', char_size=char_size, column_num=column_num, row_num=row_num):
    global sorted_index
    global current_index
    current_index = 0

    is_prompt = True
    is_image = True
    if prompt is None or prompt.replace(' ', '') == '':
        is_prompt = False
    if image is None:
        is_image = False

    if not prompt.endswith('font'):
        prompt += ' font'

    if (not is_prompt) and (not is_image):
        return [None] * (column_num * row_num)

    result_images = []
    if is_prompt and is_image:
        sorted_index, _ = query_image_and_text(image, prompt, alpha, font_db=font_db, model=model)
        for i in range(column_num):
            for j in range(row_num):
                font_path = target_font_paths[sorted_index[i * row_num + j]]
                font = ImageFont.truetype(font_path, char_size)
                image = create_image(sample_text, font)
                result_images.append(image)

        # print top5 font names
        print('top5 font names')
        for i in range(10):
            font_path = target_font_paths[sorted_index[i]]
            print(font_path.split('/')[-1].split('.')[0])

        return  result_images

    if is_prompt and (not is_image):
        sorted_index, _ = query_text(prompt, font_db, model)
        for i in range(column_num):
            for j in range(row_num):
                font_path = target_font_paths[sorted_index[i * row_num + j]]
                font = ImageFont.truetype(font_path, char_size)
                image = create_image(sample_text, font)
                result_images.append(image)
        # print top5 font names
        print('top5 font names')
        for i in range(10):
            font_path = target_font_paths[sorted_index[i]]
            print(font_path.split('/')[-1].split('.')[0])
        return  result_images
    
    if (not is_prompt) and is_image:
        sorted_index, _ = query_image(image, font_db, model)
        for i in range(column_num):
            for j in range(row_num):
                font_path = target_font_paths[sorted_index[i * row_num + j]]
                font = ImageFont.truetype(font_path, char_size)
                image = create_image(sample_text, font)
                result_images.append(image)
        # print top5 font names
        print('top5 font names')
        for i in range(10):
            font_path = target_font_paths[sorted_index[i]]
            print(font_path.split('/')[-1].split('.')[0])
        return  result_images
    
    return [None] * (column_num * row_num)

def builder_next_query(command='next', sample_text='hand write', char_size=char_size, column_num=column_num, row_num=row_num):
    assert command in ['next', 'previous']
    global current_index
    if command == 'next':
        current_index += 1
        if current_index * (column_num * row_num) >= len(sorted_index):
            current_index = 0
    else:
        current_index -= 1
        if current_index < 0:
            current_index = len(sorted_index) // (column_num * row_num) - 1


    result_images = []
    for i in range(column_num):
        for j in range(row_num):
            font_path = target_font_paths[sorted_index[current_index + i * row_num + j]]
            font = ImageFont.truetype(font_path, char_size)
            image = create_image(sample_text, font)
            result_images.append(image)
    return result_images

def builder_next_query_slider(slider_value, sample_text='hand write', char_size=char_size, column_num=column_num, row_num=row_num):
    if sorted_index is None:
        return [None] * (column_num * row_num)

    assert 0 <= slider_value <= len(target_font_paths) - 1
    global current_index
    current_index = int(slider_value)

    result_images = []
    for i in range(column_num):
        for j in range(row_num):
            font_path = target_font_paths[sorted_index[current_index + i * row_num + j]]
            font = ImageFont.truetype(font_path, char_size)
            image = create_image(sample_text, font)
            result_images.append(image)
    return result_images


css = """
.input textarea {font-size: 50px !important}
"""

gr_images = []
with gr.Blocks(css=css) as demo:
    with gr.Row():
        with gr.Column(scale=2):
            text2 = gr.Text(label='Type here to preview text', value=default_text_value, interactive=True, visible=True)
            text1 = gr.Text(label='Attribute query', interactive=True, elem_classes="input")
            slider1 = gr.Slider(0, 1.0, value=0.5, step=0.01, label='balance between attribute and image queries (0: only attribute, 1: only image)', interactive=True, visible=True)
            image1 = gr.Image(label='Image query', type="pil", interactive=True)
            button1 = gr.Button(value='Refresh', visible=True)
        with gr.Column(scale=3):
            with gr.Row():
                for i in range(column_num):
                    with gr.Column():
                        for j in range(row_num):
                            image = gr.Image(label=f'best {j+i*row_num}', type="pil", interactive=True, show_label=False)
                            gr_images.append(image)
            
            """
            with gr.Row():
                button4 = gr.Button(value='previous', interactive=True)
                button3 = gr.Button(value='next', interactive=True)
            """
            slider2 = gr.Slider(0, len(target_font_paths) - 1, value=0, step=1, label='Browse more fonts', interactive=True, visible=True)
            button2 = gr.Button(value='save outputs', interactive=True, visible=False)


    slider1.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    text1.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    image1.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    text2.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    button1.click(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    button2.click(fn=save_output_buidler, inputs=gr_images, show_progress=False)
    slider2.change(fn=builder_next_query_slider, inputs=[slider2, text2], outputs=gr_images, show_progress=False)
    #button3.click(fn=builder_next_query, inputs=[button3, text2], outputs=[image2, image3], show_progress=False)
    #button4.click(fn=builder_next_query, inputs=[button4, text2], outputs=[image2, image3], show_progress=False)

demo.launch(debug=True, share=True) 

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f5a4ac1b04a7b2b1f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


top5 font names
font_1_kokugl_1
AlibabaPuHuiTi-2-35-Thin
esenapaj
myoungheihk
NagomiGokubosoGothic-ExtraLight
AlibabaPuHuiTi-2-95-ExtraBold
NotoSansJP-Black
ZenMaruGothic-Light
Kazesawa-ExtraLight
Tanugo-TTF-Round-Light
top5 font names
SourceHanSerif-SemiBold
SourceHanSerif-ExtraLight
ゆず ポップ A [M] Light
bodang-xingkai
JP_NotoSerifJP-Regular
KH-Dot-Hibiya-24
KH-Dot-Kagurazaka-12
SourceHanSerif-Regular
36
KH-Dot-Kodenmachou-16
top5 font names
SourceHanSerif-SemiBold
SourceHanSerif-ExtraLight
ゆず ポップ A [M] Light
bodang-xingkai
JP_NotoSerifJP-Regular
KH-Dot-Hibiya-24
KH-Dot-Kagurazaka-12
SourceHanSerif-Regular
36
KH-Dot-Kodenmachou-16


Traceback (most recent call last):
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/PIL/ImageFile.py", line 503, in _save
    fh = fp.fileno()
AttributeError: '_idat' object has no attribute 'fileno'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/routes.py", line 400, in run_predict
    event_data=event_data,
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/blocks.py", line 1111, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/blocks.py", line 1045, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/components.py", line 1795, in postprocess
    return processing_utils.encod

top5 font names
SourceHanSerif-SemiBold
SourceHanSerif-ExtraLight
ゆず ポップ A [M] Light
bodang-xingkai
JP_NotoSerifJP-Regular
KH-Dot-Hibiya-24
KH-Dot-Kagurazaka-12
SourceHanSerif-Regular
36
KH-Dot-Kodenmachou-16
top5 font names
SourceHanSerif-SemiBold
SourceHanSerif-ExtraLight
ゆず ポップ A [M] Light
bodang-xingkai
JP_NotoSerifJP-Regular
KH-Dot-Hibiya-24
KH-Dot-Kagurazaka-12
SourceHanSerif-Regular
36
KH-Dot-Kodenmachou-16
top5 font names
SourceHanSerif-SemiBold
SourceHanSerif-ExtraLight
ゆず ポップ A [M] Light
bodang-xingkai
JP_NotoSerifJP-Regular
KH-Dot-Hibiya-24
KH-Dot-Kagurazaka-12
SourceHanSerif-Regular
36
KH-Dot-Kodenmachou-16
top5 font names
SourceHanSerif-Heavy
KodomoRounded-Light
KH-Dot-Kagurazaka-12
ZCOOLQingKeHuangYou-Regular
jetlink-boldrightpop
站酷庆科黄油体
zhs-caonima-ti
LXGWWenKaiMono-Bold
MPLUS1p-Thin
mgentle-hks
top5 font names
幻ノにじみ明朝
Nちはやフォント+
HarmonyOS_Sans_SC_Thin
MWindyHKS-Bold
Kaisotai-Next-UP-B
KH-Dot-Kodenmachou-16
SourceHanSerif-SemiBold
SourceHanSerif-ExtraLight
Kodom

In [6]:
font_images_num = 2
def save_output_buidler(image1, image2):
    image1.save('outputs/image1.png')
    image2.save('outputs/image2.png')


default_text_value = 'hand write'
#default_text_value = 'Sushi.\nStill your best bet\nfor intestinal worms.'
sorted_index = None
current_index = 0
column_num = 2
row_num = 3
def builder_query(prompt, image, alpha=1.0, sample_text='hand write', char_size=char_size, column_num=column_num, row_num=row_num):
    global sorted_index
    global current_index
    current_index = 0

    is_prompt = True
    is_image = True
    if prompt is None or prompt.replace(' ', '') == '':
        is_prompt = False
    if image is None:
        is_image = False

    if not prompt.endswith('font'):
        prompt += ' font'

    if (not is_prompt) and (not is_image):
        return None, None

    result_images = []
    if is_prompt and is_image:
        sorted_index, _ = query_image_and_text(image, prompt, alpha, font_db=font_db, model=model)
        for i in range(column_num):
            for j in range(row_num):
                font_path = target_font_paths[sorted_index[i * row_num + j]]
                font = ImageFont.truetype(font_path, char_size)
                image = create_image(sample_text, font)
                result_images.append(image)

        # print top5 font names
        print('top5 font names')
        for i in range(9):
            font_path = target_font_paths[sorted_index[i]]
            # print(font_path.split('/')[-1].split('.')[0])

        return  result_images

    if is_prompt and (not is_image):
        sorted_index, _ = query_text(prompt, font_db, model)
        for i in range(column_num):
            for j in range(row_num):
                font_path = target_font_paths[sorted_index[i * row_num + j]]
                font = ImageFont.truetype(font_path, char_size)
                image = create_image(sample_text, font)
                result_images.append(image)
        return  result_images
    
    if (not is_prompt) and is_image:
        sorted_index, _ = query_image(image, font_db, model)
        for i in range(column_num):
            for j in range(row_num):
                font_path = target_font_paths[sorted_index[i * row_num + j]]
                font = ImageFont.truetype(font_path, char_size)
                image = create_image(sample_text, font)
                result_images.append(image)
        return  result_images
    
    return [None] * (column_num * row_num)

def builder_next_query(command='next', sample_text='hand write', char_size=char_size, column_num=column_num, row_num=row_num):
    assert command in ['next', 'previous']
    global current_index
    if command == 'next':
        current_index += 1
        if current_index * (column_num * row_num) >= len(sorted_index):
            current_index = 0
    else:
        current_index -= 1
        if current_index < 0:
            current_index = len(sorted_index) // (column_num * row_num) - 1


    result_images = []
    for i in range(column_num):
        for j in range(row_num):
            font_path = target_font_paths[sorted_index[current_index + i * row_num + j]]
            font = ImageFont.truetype(font_path, char_size)
            image = create_image(sample_text, font)
            result_images.append(image)
    return result_images

def builder_next_query_slider(slider_value, sample_text='hand write', char_size=char_size, column_num=column_num, row_num=row_num):
    if sorted_index is None:
        return [None] * (column_num * row_num)

    assert 0 <= slider_value <= len(target_font_paths) - 1
    global current_index
    current_index = int(slider_value)

    result_images = []
    for i in range(column_num):
        for j in range(row_num):
            font_path = target_font_paths[sorted_index[current_index + i * row_num + j]]
            font = ImageFont.truetype(font_path, char_size)
            image = create_image(sample_text, font)
            result_images.append(image)
    return result_images


gr_images = []
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=2):
            text2 = gr.Text(label='sample text', value=default_text_value, interactive=True)
            text1 = gr.Text(label='attribute query', interactive=True)
            slider1 = gr.Slider(0, 1.0, value=0.5, step=0.1, label='balance between attribute and image queries (0: only attribute, 1: only image)', interactive=True, visible=True)
            image1 = gr.Image(label='image query', type="pil", interactive=True)
            button1 = gr.Button(value='refresh', interactive=True)
        with gr.Column(scale=3):
            with gr.Row():
                for i in range(column_num):
                    with gr.Column():
                        for j in range(row_num):
                            image = gr.Image(label=f'best {j+i*row_num}', type="pil", interactive=True, show_label=False)
                            gr_images.append(image)
            
            """
            with gr.Row():
                button4 = gr.Button(value='previous', interactive=True)
                button3 = gr.Button(value='next', interactive=True)
            """
            slider2 = gr.Slider(0, len(target_font_paths) - 1, value=0, step=1, label='display index', interactive=True)
            button2 = gr.Button(value='save outputs', interactive=True)


    slider1.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    text1.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    image1.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    text2.change(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    button1.click(fn=builder_query, inputs=[text1, image1, slider1, text2], outputs=gr_images, show_progress=False)
    button2.click(fn=save_output_buidler, inputs=gr_images, show_progress=False)
    slider2.change(fn=builder_next_query_slider, inputs=[slider2, text2], outputs=gr_images, show_progress=False)
    #button3.click(fn=builder_next_query, inputs=[button3, text2], outputs=[image2, image3], show_progress=False)
    #button4.click(fn=builder_next_query, inputs=[button4, text2], outputs=[image2, image3], show_progress=False)

demo.launch(debug=True, share=True) 

/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/utils.py:875: UserWarning: Expected 2 arguments for function <function save_output_buidler at 0x7fa663a63a70>, received 6.
  f"Expected {max_args} arguments for function {fn}, received {arg_count}."
/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/utils.py:883: UserWarning: Expected maximum 2 arguments for function <function save_output_buidler at 0x7fa663a63a70>, received 6.
  f"Expected maximum {max_args} arguments for function {fn}, received {arg_count}."


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://467cb1a2a62c50e559.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/blocks.py", line 1020, in postprocess_data
    if predictions[i] is components._Keywords.FINISHED_ITERATING:
IndexError: tuple index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/routes.py", line 400, in run_predict
    event_data=event_data,
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/blocks.py", line 1111, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/blocks.py", line 1025, in postprocess_data
    f"Number of output components does not match number of values returned from from function {block_fn.name}"
ValueError: Number of output components does not match number of v

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://467cb1a2a62c50e559.gradio.live


# list method

In [14]:
target_font_paths = all_gwfont_paths
target_font_paths = font_paths
#target_font_paths = cj_font_paths

char_size = 200
column_num = 4
row_num = 3
image_num_per_page = column_num * row_num
images = []
print(len(target_font_paths) // image_num_per_page)



def builder_list_query(slider_value, sample_text='hand write', char_size=char_size):
    assert 0 <= slider_value <= len(target_font_paths) // image_num_per_page
    start_index = int(slider_value * image_num_per_page)
    end_index = int((slider_value + 1) * image_num_per_page) if int((slider_value + 1) * image_num_per_page) < len(target_font_paths) else len(target_font_paths)

    tmp_images = []
    for i in range(start_index, end_index):
        font_path = target_font_paths[i]
        font = ImageFont.truetype(font_path, char_size)
        image = create_image(sample_text, font)
        tmp_images.append(image)
    for i in range(image_num_per_page - len(tmp_images)):
        #tmp_images.append(Image.new('RGB', (char_size, char_size), (255, 255, 255)))
        tmp_images.append(None)

    # return unpacked list
    return tmp_images


default_images = builder_list_query(0)
with gr.Blocks() as demo:
    text1 = gr.Text(label='sample text', value='hand write', interactive=True)
    with gr.Row():
        for i in range(column_num):
            with gr.Column():
                for j in range(row_num):
                    image = gr.Image(value=default_images[i*row_num+j], type="pil", interactive=True, show_label=False)
                    #image = gr.Image(value=default_images[i*row_num+j], type="pil", interactive=False, show_label=False)
                    images.append(image)
    slider1 = gr.Slider(0, len(target_font_paths) // image_num_per_page, value=0, step=1, label='display index', interactive=True)
    button1 = gr.Button(value='refresh', interactive=True)
    slider1.change(fn=builder_list_query, inputs=[slider1, text1], outputs=images, show_progress=False)
    text1.change(fn=builder_list_query, inputs=[slider1, text1], outputs=images, show_progress=False)

    

demo.launch(debug=True, share=True)


16
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://dfb32c67f7ceb0c13e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/PIL/ImageFile.py", line 503, in _save
    fh = fp.fileno()
AttributeError: '_idat' object has no attribute 'fileno'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/routes.py", line 400, in run_predict
    event_data=event_data,
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/blocks.py", line 1111, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/blocks.py", line 1045, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/home/yuki/anaconda3/envs/dgfont-env/lib/python3.7/site-packages/gradio/components.py", line 1795, in postprocess
    return processing_utils.encod

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dfb32c67f7ceb0c13e.gradio.live


In [18]:
3 // 5

0